Search using Top Charts and HipHopCharts, created connected graph of artists

artist can collaborate with other artists, so we can consider Ingoing and Outgoing links for page rank

Create visual circles describing 

Graph of words for different genres: https://towardsdatascience.com/getting-started-with-graph-analysis-in-python-with-pandas-and-networkx-5e2d2f82f18e

https://networkx.github.io/
https://plot.ly/python/3d-network-graph/

Max distance from one node to another
Avg distance from one node to another
Pagerank


artist = domain
song = document
collaborators = outgoing links / incoming links
lyrics = docuemnt text

In [1]:
from pymongo import MongoClient

# DB CONFIG
client = MongoClient('mongo', 27017)
db = client.music_db

In [2]:
db.collection_names()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['pop-songs',
 'latin-songs',
 'top_songs',
 'country-songs',
 'rock-songs',
 'songs',
 'jazz-songs',
 'all-songs',
 'christian-songs',
 'rap-song']

In [3]:
songs = list(db['songs'].find())
songs[1]

{'_id': ObjectId('5ba78864539667005f93a706'),
 'title': 'Girls Like You',
 'artist': 'Maroon 5 Featuring Cardi B',
 'peakPos': 94,
 'lastPos': 0,
 'weeks': 0,
 'rank': 94,
 'isNew': True,
 'identifier': "'Girls Like You' by Maroon 5 Featuring Cardi B",
 'lyrics': {'error': 'Lyric no found, try again later.'}}

In [4]:
# Filter Collaboration Songs 
SEPARATORS = ['&', 'Featuring', 'With', ',', 'Feat.']

def is_collaboration(artist):
    return any([word in artist for word in SEPARATORS])

collaborations = songs #[song for song in songs if is_collaboration(song['artist'])]

In [5]:

import re
from typing import List
import re

def clean(artists: List) -> List:
    artists = [artist.strip() for artist in artists]
    alphanum = re.compile(r'[^\s\w]+')
    artists = [alphanum.sub('', artist) for artist in artists]
    return artists

def separate(artist: str) -> List:
    return re.split('|'.join(SEPARATORS), artist)

def add_collaborators_to_songs(collaborators, songs):
    for collabs, song in zip(collaborators, songs):
        song['collaborators'] = collabs
    return songs

collaborators = [song['artist'] for song in collaborations]
collaborators = map(separate, collaborators)
collaborators = map(clean, collaborators)
collaborators = list(collaborators)        
collaborations = add_collaborators_to_songs(collaborators, collaborations)
collaborators[:2]


[['Drake'], ['Maroon 5', 'Cardi B']]

In [6]:
collaborations[0]['peakPos']

6

In [7]:
import networkx as nx
G = nx.DiGraph()

In [8]:
from functools import reduce
from operator import add

def unique_artists(songs):
    artists = [song['collaborators'] for song in songs]   # List of lists
    artists = reduce(add, collaborators)
    artists = [artist.strip() for artist in artists]
    return set(artists)

nodes = unique_artists(collaborations)
G.add_nodes_from(nodes)

In [9]:
from typing import List, Tuple

def make_edges(songs) -> List[Tuple]:
    """ Returns tuples, which creates an edge between collaborators """
    edges = []
    for song in songs:
        primary, *collabs = song['collaborators']
        if collabs:
            edges.append((primary, *collabs))
    return edges

def add_edges(graph: nx.Graph, edges: List[Tuple]):
    for primary, *collabs in edges:
        for collab in collabs:
            graph.add_edge(primary, collab)
            
    
edges = make_edges(collaborations)
add_edges(G, edges)

In [10]:
G.edges

OutEdgeView([('Monica', 'Dem Franchize Boyz'), ('Lenny Kravitz', 'JayZ'), ('Tim McGraw', 'Faith Hill'), ('Tim McGraw', 'Catherine Dunn'), ('Tim McGraw', 'Taylor Swift'), ('Tyler', 'The Creator'), ('Justin Timberlake', 'Chris Stapleton'), ('Justin Timberlake', 'JAY Z'), ('Justin Timberlake', 'Matt Morris'), ('Justin Timberlake', 'Charlie Sexton'), ('Justin Timberlake', 'TI'), ('Kid Rock', 'Sheryl Crow'), ('Jay Sean', 'Pitbull'), ('Jay Sean', 'Lil Wayne'), ('Jay Sean', 'Nicki Minaj'), ('Jay Sean', 'Sean Paul'), ('Jay Sean', 'Lil Jon'), ('Bandit Gang Marco', 'Dro'), ('Bone Crusher', 'Killer Mike'), ('Bone Crusher', 'TI'), ('Chris Jamison', 'Adam Levine'), ('Sevyn Streeter', 'Chris Brown'), ('Missy Elliott', 'Lamb'), ('Missy Elliott', 'Pharrell Williams'), ('Missy Elliott', 'Ciara'), ('Missy Elliott', 'Fat Man Scoop'), ('Ashley Tisdale', 'Lucas Grabeel'), ('Jay Z Kanye West', 'Frank Ocean'), ('Jay Z Kanye West', 'Otis Redding'), ('Daddy Yankee', 'Fergie'), ('LinManuel Miranda', 'Ben Platt'

## Draw Graph

In [11]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py

options = {}

def assign_positions(graph: nx.Graph):
    positions = nx.spring_layout(G)
    nx.set_node_attributes(G, positions, 'pos')
    return G
    
assign_positions(G)

In [12]:

def plot_graph(G):

    pos = nx.get_node_attributes(G, 'pos')

    dmin = 1
    ncenter = 'Drake'
    for n in pos:
        x, y= pos[n]
        d=(x-0.5)**2+(y-0.5)**2
        if d<dmin:
            ncenter=n
            dmin=d

    p = nx.single_source_shortest_path_length(G,ncenter)

    # 

    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])

    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))

    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])

    adj_length = len(list(G.adjacency()))

    from IPython.display import display, clear_output

    i = 0
    for node, adjacencies in enumerate(G.adjacency()):
        node_trace['marker']['color']+=tuple([len(adjacencies[1])])
        name = adjacencies[0]
        node_info = f"""
                        {name} | 
                        # of outgoing connections: {len(adjacencies[1])} | 
                        # of incoming connections: {G.in_degree(name)}
                        """
        node_trace['text'] += tuple([node_info])
        i +=1
        if i % 10 == 0:
            clear_output(wait=True)
            print(f'{i}/{adj_length}', end=" ")

    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title='<br>Artists Connections',
                    titlefont=dict(size=16),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

    return py.iplot(fig, filename='networkx')
    


In [13]:
plot_graph(G)

2750/2753 

/opt/conda/lib/python3.6/site-packages/plotly/tools.py:1428: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



In [14]:
# Sort by outgoing connections
# Sort by incoming connections

In [15]:
def sorted_nodes(g: nx.Graph, out_degree=False):
    """in_degree is False"""
    sort_type = G.out_degree if out_degree else G.in_degree
    sorted_n = sorted(G.nodes, key=sort_type, reverse=True)
    return [(node, sort_type(node)) for node in sorted_n]

in_degrees = sorted_nodes(G)
out_degrees = sorted_nodes(G, out_degree=True)

In [16]:
from pprint import pprint
from pandas import DataFrame 
import pandas as pd
from operator import itemgetter

df_in = DataFrame(in_degrees, columns=['Artist', 'In_Degree'])
df_out = DataFrame(out_degrees, columns=['Artist', 'Out_Degree'])
df = pd.merge(df_in, df_out, on=['Artist'])
df['Total_Degree'] = df['In_Degree'] + df['Out_Degree']
df = df.sort_values('Out_Degree', ascending=False)
df.head()
df[:30]

,Artist,In_Degree,Out_Degree,Total_Degree
131,DJ Khaled,4,37,41
3,Drake,33,35,68
0,Lil Wayne,51,32,83
4,Kanye West,26,31,57
2,Chris Brown,34,23,57
177,David Guetta,3,22,25
30,Pitbull,11,22,33
57,Nelly,7,21,28
156,Meek Mill,3,19,22
10,JayZ,20,19,39


In [17]:
def subgraph(G, n_keep=3):
    to_keep = []
    for i, row in df.iterrows():
        if row['In_Degree'] >= n_keep or row['Out_Degree'] >= n_keep:
            to_keep.append(row['Artist'])  
            
    return G.subgraph(to_keep)

sub_g = subgraph(G, n_keep=5)
assign_positions(sub_g)
plot_graph(sub_g)

# 270 nodes

170/179 

/opt/conda/lib/python3.6/site-packages/plotly/tools.py:1428: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



In [18]:
# Apply algorithms from https://networkx.github.io/documentation/stable/reference/algorithms/index.html
# Make weight the best ranking of the song
# Distribute page rank equally among artists
import networkx.algorithms as nxa

sub_g = subgraph(G, n_keep=10)
#nxa.average_shortest_path_length(H)
sc = list(nxa.strongly_connected_components(sub_g))
nodes = max(sc, key=len)
SC = sub_g.subgraph(nodes)
plot_graph(SC)

50/57 

/opt/conda/lib/python3.6/site-packages/plotly/tools.py:1428: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



In [19]:
nxa.average_shortest_path_length(SC.to_undirected())

2.006265664160401

In [25]:
from functools import reduce
from operator import or_

weak_components = list(nxa.weakly_connected_components(G))
weak_component = max(weak_components, key=len)
WK = G.subgraph(weak_component)
nxa.is_weakly_connected(WK)
plot_graph(WK)


1230/1233 

/opt/conda/lib/python3.6/site-packages/plotly/tools.py:1428: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



In [21]:
nxa.average_shortest_path_length(WK.to_undirected())

4.8828298627568705

In [22]:
from collections import Counter
pagerank = nxa.pagerank(G)
pagerank = Counter(pagerank).most_common()
DataFrame(pagerank, columns=["Artist", "Page_Rank"])[:50]

,Artist,Page_Rank
0,Lil Wayne,0.006595
1,Nicki Minaj,0.006166
2,Drake,0.005719
3,Kanye West,0.005301
4,Chris Brown,0.004218
5,Ludacris,0.004035
6,Akon,0.004012
7,Future,0.003639
8,JayZ,0.003621
9,2 Chainz,0.003217


In [ ]:
# Use song score in the calcluation